In [1]:
import os
import sys
import pandas as pd
import torch
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from machine_learning.preprocess_ml import DataPipeline
from machine_learning.train_models import TransformerFineTuner
    
# load data
df_a = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\training_data.csv")
mapping = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\topic_mapping.csv")

df = df_a.merge(right=mapping, how='inner', right_on='topic_sortedc_str', left_on='topic_sortedc_str')[['cleaned_text', 'topic_hl']]
df['topic_hl_encoded'] = df['topic_hl'].astype("category").cat.codes


# prepare data sets
pipeline = DataPipeline(df, target_col='topic_hl_encoded', text='cleaned_text', model='cardiffnlp/twitter-roberta-base')
train_loader = pipeline.get_train_data()
val_loader = pipeline.get_val_data()
test_loader = pipeline.get_test_data()

C:\Users\johna\anaconda3\envs\twitter-analytics-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
test_loader.s